# Quantopian
* Review of Quantopian
* Basic Algorithm Methods
* Building Trading Algorithms
* Trading Algorithm Exercise
* Quantopian Pipelines

https://www.zipline.io/

In [3]:
def initialize(context):
    context.amzn = sid(16841)
    context.ibm = sid(3766)
    
    schedule_function(rebalance,
                      date_rules.every_day(),
                      time_rules.market_open())
    
    schedule_function(record_vars,
                      date_rules.every_day(),
                      time_rules.market_close())
                      
    
def rebalance(context,data):
    order_target_percent(context.amzn,0.5)
    order_target_percent(context.ibm,-0.5)
    
def record_vars(context,data):
    record(amzn_close =data.current(context.amzn,'close'))
    record(ibm_close_jelly=data.current(context.ibm,'close'))
                                 

In [5]:
#https://www.investopedia.com/terms/d/drawdown.asp

"""
This is a template algorithm on Quantopian for you to adapt and fill in.
"""
import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS


def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    # Rebalance every day, 1 hour after market open.
    algo.schedule_function(
        rebalance,
        algo.date_rules.every_day(),
        algo.time_rules.market_open(hours=1),
    )

    # Record tracking variables at the end of each day.
    algo.schedule_function(
        record_vars,
        algo.date_rules.every_day(),
        algo.time_rules.market_close(),
    )

    # Create our dynamic stock selector.
    algo.attach_pipeline(make_pipeline(), 'pipeline')


def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """

    # Base universe set to the QTradableStocksUS
    base_universe = QTradableStocksUS()

    # Factor of yesterday's close price.
    yesterday_close = USEquityPricing.close.latest

    pipe = Pipeline(
        columns={
            'close': yesterday_close,
        },
        screen=base_universe
    )
    return pipe


def before_trading_start(context, data):
    """
    Called every day before market open.
    """
    context.output = algo.pipeline_output('pipeline')

    # These are the securities that we are interested in trading each day.
    context.security_list = context.output.index


def rebalance(context, data):
    """
    Execute orders according to our schedule_function() timing.
    """
    pass


def record_vars(context, data):
    """
    Plot variables at the end of each day.
    """
    pass


def handle_data(context, data):
    """
    Called every minute.
    """
    pass

In [9]:
import numpy as np

# initializie - schedule function
def initialize(context):
    
    schedule_function(check_pairs,
                     date_rules.every_day(),
                     time_rules.market_close(minutes=60))
    
    context.aa = sid(45971)
    context.ual = sid(28051)
    
    context.long_on_spread= False
    context.shorting_spread = False
        
    
    
#check_pairs:
def check_pairs(context,data):
    
    aa = context.aa
    ual = context.ual
    
    prices = data.history([aa,ual],'price',30,'1d')
    
    # Current timestamp in backtest
    short_prices = prices.iloc[-1:]
    
    # Everything is in the context of the spread
    ma_30 = np.mean(prices[aa] - prices[ual])
    std_30 = np.std(prices[aa]- prices[ual])
    
    # current price
    ma_1 = np.mean(short_prices[aa] - short_prices[ual])
    
    # IF EVER UNCLEAR PRINT THEM OUT
    
    if std_30 > 0:
        zscore = (ma_1 - ma_30) / std_30
        
        # Two entry cases above threshold or below:        
        if zscore > 0.5 and not context.shorting_spread:
            # Spread AA - UAL prices
            # AA is currently over valued and will regress
            order_target_percent(aa,-0.5)
            order_target_percent(ual,0.5)
            context.shorting_spread = True
            context.long_on_spread = False
        elif zscore < 1.0 and not context.long_on_spread:
            order_target_percent(aa,0.5)
            order_target_percent(ual,-0.5)
            context.shorting_spread = False
            context.long_on_spread = True
        elif abs(zscore) < 0.1:
            # Exit position as we get to 0
            order_target_percent(aa,0)
            order_target_percent(ual,0)
            context.shorting_spread = False
            context.long_on_spread = False
         
        record(z_score = zscore)
                                 
                           
            
            
            
            
    